<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных-в-локальную-файловую-систему" data-toc-modified-id="Загрузка-данных-в-локальную-файловую-систему-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных в локальную файловую систему</a></span></li><li><span><a href="#Загрузка-данных-на-hdfs" data-toc-modified-id="Загрузка-данных-на-hdfs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных на hdfs</a></span></li><li><span><a href="#Чтение-данных-из-csv-и-сохранение-в-parquet-формат" data-toc-modified-id="Чтение-данных-из-csv-и-сохранение-в-parquet-формат-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Чтение данных из csv и сохранение в parquet формат</a></span></li><li><span><a href="#min/max" data-toc-modified-id="min/max-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>min/max</a></span></li><li><span><a href="#where" data-toc-modified-id="where-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>where</a></span></li><li><span><a href="#z.textbox" data-toc-modified-id="z.textbox-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>z.textbox</a></span></li><li><span><a href="#Сохранение-в-hive-metastore" data-toc-modified-id="Сохранение-в-hive-metastore-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Сохранение в hive metastore</a></span></li><li><span><a href="#Добавление-колонки-+-сохранение-по-партициям" data-toc-modified-id="Добавление-колонки-+-сохранение-по-партициям-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Добавление колонки + сохранение по партициям</a></span></li><li><span><a href="#Группировка" data-toc-modified-id="Группировка-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Группировка</a></span></li></ul></div>

## Загрузка данных в локальную файловую систему

In [ ]:
%sh

cd data
rm -rf data_nov.csv.zip
curl http://37.139.43.86/data-nov > data_nov.csv


ls -lh ./

## Загрузка данных на hdfs

In [ ]:
%sh

hdfs dfs -mkdir data
hdfs dfs -put data/data_nov.csv data

In [ ]:
%sh

hdfs dfs -du -h data

In [ ]:
%pyspark

spark.sql("show databases").show()

## Чтение данных из csv и сохранение в parquet формат

In [ ]:
%pyspark

# schema = "event_time timestamp, event_type string, product_id long, category_id long, category_code string, brand string, price float, user_id long, user_session string"

df = spark.read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv("data/data_nov.csv")

df.printSchema()

df \
.write.mode("overwrite") \
.format("parquet") \
.save("data/data_nov.parquet")

In [ ]:
%pyspark
df.show()

In [ ]:
%sh

hdfs dfs -du -h data

In [ ]:
%sh

hdfs dfs -du -h data/data_nov.parquet

In [ ]:
%pyspark

spark.read.parquet("data/data_nov.parquet").coalesce(1).write.mode("overwrite").parquet("data/data_nov_coalesce.parquet")

In [ ]:
%sh
hdfs dfs -du -h data/data_nov_coalesce.parquet

In [ ]:
%pyspark

events_df = spark.read.parquet("data/data_nov_coalesce.parquet")
events_df.printSchema()
events_df.show(100)

## min/max

In [ ]:
%pyspark

events_df.select(f.min("event_time"), f.max("event_time")).show(truncate=False)

## where

In [ ]:
%pyspark

events_df.where("brand = 'intel'").show()

## z.textbox

In [ ]:
%pyspark

print("Hello " + z.textbox("name"))

## Сохранение в hive metastore

In [ ]:
%pyspark

spark.sql("create database src_data")
events_df.write.saveAsTable("src_data.events")

In [ ]:
%sh

hdfs dfs -du -h /apps/hive/warehouse

In [ ]:
%sh

hdfs dfs -du -h /apps/hive/warehouse/src_data.db/events

## Добавление колонки + сохранение по партициям

In [ ]:
%pyspark
import pyspark.sql.functions as f


spark.sql("create database market")
spark.table("src_data.events") \
.withColumn("date", f.to_date("event_time")) \
.write.partitionBy("date").saveAsTable("market.events")

In [ ]:
%sh

hdfs dfs -du -h /apps/hive/warehouse/market.db/events

In [ ]:
%sh

hdfs dfs -du -h /apps/hive/warehouse/market.db/events/date=2019-11-03

## Группировка

In [ ]:
%pyspark
import pyspark.sql.functions as f

events_df \
    .groupBy("event_type") \
    .agg(
        f.count("*").alias("count")
        ) \
    .orderBy(f.col("count").desc()) \
    .show(truncate=False)